# Readiness and Liveness Probe

## 参考文献

* *https://kubernetes.io/docs/tasks/configure-pod-container/configure-liveness-readiness-startup-probes/*

## 就绪探测

kubelet就绪探测成功后，容器就可以接收实际的流量了。

```
The kubelet uses readiness probes to know when a container is ready to start accepting traffic. A Pod is considered ready when all of its containers are ready. One use of this signal is to control which Pods are used as backends for Services. When a Pod is not ready, it is removed from Service load balancers.
```

## 存活探测

kubelet存活探测失败，就会重启容器。

```
The kubelet uses liveness probes to know when to restart a container. For example, liveness probes could catch a deadlock, where an application is running, but unable to make progress. Restarting a container in such a state can help to make the application more available despite bugs.
```

## 配置探测器

* `initialDelaySeconds`：容器启动后要等待多少秒后存活和就绪探测器才被初始化，默认是 0 秒，最小值是 0。
* `periodSeconds`：执行探测的时间间隔（单位是秒）。默认是 10 秒。最小值是 1。
* `timeoutSeconds`：探测的超时后等待多少秒。默认值是 1 秒。最小值是 1。
* `successThreshold`：探测器在失败后，被视为成功的最小连续成功数。默认值是 1。 存活和启动探测的这个值必须是 1。最小值是 1。
* `failureThreshold`：当探测失败时，Kubernetes 的重试次数。 存活探测情况下的放弃就意味着重新启动容器。 就绪探测情况下的放弃 Pod 会被打上未就绪的标签。默认值是 3。最小值是 1。

### HTTP探测

* `host`：连接使用的主机名，默认是 Pod 的 IP。也可以在 HTTP 头中设置 “Host” 来代替。
* `scheme` ：用于设置连接主机的方式（HTTP 还是 HTTPS）。默认是 HTTP。
* `path`：访问 HTTP 服务的路径。默认值为 "/"。
* `httpHeaders`：请求中自定义的 HTTP 头。HTTP 头字段允许重复。
* `port`：访问容器的端口号或者端口名。如果数字必须在 1 ～ 65535 之间。

```yaml
livenessProbe:
  httpGet:
    path: /healthz
    port: liveness-port
    httpHeaders:
      - name: Accept
        value: application/json

```

### TCP探测

```yaml
livenessProbe:
  tcpSocket:
    port: 8080
  initialDelaySeconds: 15
  periodSeconds: 20
```

示例：

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: goproxy
  labels:
    app: goproxy
spec:
  containers:
  - name: goproxy
    image: k8s.gcr.io/goproxy:0.1
    ports:
    - containerPort: 8080
    readinessProbe:
      tcpSocket:
        port: 8080
      initialDelaySeconds: 5
      periodSeconds: 10
    livenessProbe:
      tcpSocket:
        port: 8080
      initialDelaySeconds: 15
      periodSeconds: 20

```